Step 1: Kaggle download + imports

In [23]:
!pip -q install kaggle

import os, json, shutil, random
from pathlib import Path

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print("TensorFlow:", tf.__version__)
print("GPU:", tf.config.list_physical_devices("GPU"))

TensorFlow: 2.19.0
GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


Step 2:  Download CelebA from Kaggle

In [24]:
!pip -q install kaggle

import os, json
from pathlib import Path
from getpass import getpass

KAGGLE_USERNAME = input("Kaggle username: ")
KAGGLE_KEY = getpass("Kaggle API key (hidden): ")

kaggle_dir = Path.home() / ".kaggle"
kaggle_dir.mkdir(parents=True, exist_ok=True)

kaggle_json_path = kaggle_dir / "kaggle.json"
with open(kaggle_json_path, "w") as f:
    json.dump({"username": KAGGLE_USERNAME, "key": KAGGLE_KEY}, f)

os.chmod(kaggle_json_path, 0o600)
print("✅ kaggle.json created at:", kaggle_json_path)

data_root = Path("/content/celeba_kaggle")
data_root.mkdir(parents=True, exist_ok=True)

!kaggle datasets download -d jessicali9530/celeba-dataset -p {data_root} --unzip
!ls -lah {data_root}

Kaggle username: prithilasaha
Kaggle API key (hidden): ··········
✅ kaggle.json created at: /root/.kaggle/kaggle.json
Dataset URL: https://www.kaggle.com/datasets/jessicali9530/celeba-dataset
License(s): other
100% 1.33G/1.33G [00:48<00:00, 4.14MB/s]
100% 1.33G/1.33G [00:48<00:00, 29.3MB/s]
total 45M
drwxr-xr-x 3 root root 4.0K Jan  7 21:15 .
drwxr-xr-x 1 root root 4.0K Jan  7 21:02 ..
-rw-r--r-- 1 root root 3.3M Jan  7 20:37 identity_CelebA.txt
drwxr-xr-x 3 root root 4.0K Jan  7 20:35 img_align_celeba
-rw-r--r-- 1 root root  24M Jan  7 21:15 list_attr_celeba.csv
-rw-r--r-- 1 root root 5.2M Jan  7 21:15 list_bbox_celeba.csv
-rw-r--r-- 1 root root 2.8M Jan  7 21:15 list_eval_partition.csv
-rw-r--r-- 1 root root 9.5M Jan  7 21:15 list_landmarks_align_celeba.csv


Step 3: Locate image folder + identity labels

In [25]:
from pathlib import Path
import os

data_root = Path("/content/celeba_kaggle")

# Find image folder
img_dir = None
for cand in ["img_align_celeba", "img_align_celeba/img_align_celeba"]:
    hits = list(data_root.rglob(cand))
    if hits:
        img_dir = hits[0]
        break

print("img_dir:", img_dir)
if img_dir is None:
    raise FileNotFoundError("Could not find img_align_celeba folder inside your dataset.")

# Find identity file
identity_path = None
hits = list(data_root.rglob("identity_CelebA.txt"))
if hits:
    identity_path = hits[0]

if identity_path is None:
    identity_path = data_root / "identity_CelebA.txt"
    !wget -q -O "{identity_path}" https://raw.githubusercontent.com/Golbstein/keras-face-recognition/master/identity_CelebA.txt
    print("Downloaded identity file to:", identity_path)

# Sanity check
if not identity_path.exists() or identity_path.stat().st_size < 1000:
    raise FileNotFoundError("identity_CelebA.txt is missing or too small; download failed.")

print("identity_path:", identity_path)
print("identity file size (bytes):", identity_path.stat().st_size)

img_dir: /content/celeba_kaggle/img_align_celeba
identity_path: /content/celeba_kaggle/identity_CelebA.txt
identity file size (bytes): 3424458


Step 4: Dataset Splitting

In [44]:
import shutil, random
from pathlib import Path

SEED = 42
random.seed(SEED)

src_root = Path("/content/reduced_celeba")
dst_root = Path("/content/reduced_celeba_split")

TRAIN_PER_CLASS = 20
VAL_PER_CLASS = 5

if dst_root.exists():
    shutil.rmtree(dst_root)
(dst_root / "train").mkdir(parents=True, exist_ok=True)
(dst_root / "val").mkdir(parents=True, exist_ok=True)

class_dirs = sorted([d for d in src_root.iterdir() if d.is_dir()])
print("Classes found:", len(class_dirs))

for cls_dir in class_dirs:
    imgs = sorted(list(cls_dir.glob("*.jpg")))
    random.shuffle(imgs)

    if len(imgs) < TRAIN_PER_CLASS + VAL_PER_CLASS:
        raise ValueError(f"{cls_dir.name} has only {len(imgs)} images. Need {TRAIN_PER_CLASS+VAL_PER_CLASS}.")

    train_imgs = imgs[:TRAIN_PER_CLASS]
    val_imgs = imgs[TRAIN_PER_CLASS:TRAIN_PER_CLASS + VAL_PER_CLASS]

    (dst_root / "train" / cls_dir.name).mkdir(parents=True, exist_ok=True)
    (dst_root / "val" / cls_dir.name).mkdir(parents=True, exist_ok=True)

    for p in train_imgs:
        shutil.copy2(p, dst_root / "train" / cls_dir.name / p.name)
    for p in val_imgs:
        shutil.copy2(p, dst_root / "val" / cls_dir.name / p.name)

print("✅ Done.")
print("Train images:", sum(1 for _ in (dst_root/'train').rglob('*.jpg')))
print("Val images:", sum(1 for _ in (dst_root/'val').rglob('*.jpg')))

Classes found: 10
✅ Done.
Train images: 200
Val images: 50


Step 5: Dataset Loading and Preprocessing

In [45]:
import tensorflow as tf

IMG_SIZE = (160, 160)
BATCH_SIZE = 16
SEED = 42

train_ds = tf.keras.utils.image_dataset_from_directory(
    "/content/reduced_celeba_split/train",
    label_mode="categorical",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=SEED,
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    "/content/reduced_celeba_split/val",
    label_mode="categorical",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False,
)

NUM_CLASSES = len(train_ds.class_names)
print("NUM_CLASSES:", NUM_CLASSES)

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(AUTOTUNE)
val_ds = val_ds.cache().prefetch(AUTOTUNE)


Found 200 files belonging to 10 classes.
Found 50 files belonging to 10 classes.
NUM_CLASSES: 10


Step 6: Model Architecture and Training

In [46]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

aug = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.02),
    layers.RandomZoom(0.05),
], name="aug")

def ln_relu(x, name):
    x = layers.LayerNormalization(epsilon=1e-6, name=f"{name}_ln")(x)
    return layers.ReLU(6., name=f"{name}_relu")(x)

def dw_ln_block(x, out_ch, stride, name):
    in_ch = x.shape[-1]
    shortcut = x

    x = layers.DepthwiseConv2D(3, strides=stride, padding="same", use_bias=False, name=f"{name}_dw")(x)
    x = ln_relu(x, f"{name}_a")

    x = layers.Conv2D(out_ch, 1, padding="same", use_bias=False, name=f"{name}_pw")(x)
    x = ln_relu(x, f"{name}_b")

    if stride == 1 and in_ch == out_ch:
        x = layers.Add(name=f"{name}_add")([x, shortcut])
    return x

def build_task1(input_shape, num_classes):
    inp = keras.Input(shape=input_shape)
    x = aug(inp)
    x = layers.Rescaling(1./255)(x)

    x = layers.Conv2D(32, 3, strides=2, padding="same", use_bias=False)(x)
    x = ln_relu(x, "stem")

    x = dw_ln_block(x, 64,  1, "b1")
    x = dw_ln_block(x, 128, 2, "b2")
    x = dw_ln_block(x, 128, 1, "b3")
    x = dw_ln_block(x, 256, 2, "b4")
    x = dw_ln_block(x, 256, 1, "b5")
    x = dw_ln_block(x, 256, 1, "b6")

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.1)(x)
    out = layers.Dense(num_classes, activation="softmax")(x)
    return keras.Model(inp, out, name="task1_scratch")

model1 = build_task1(IMG_SIZE + (3,), NUM_CLASSES)
model1.summary()

steps = tf.data.experimental.cardinality(train_ds).numpy()
EPOCHS = 80
lr = keras.optimizers.schedules.CosineDecay(1e-3, steps * EPOCHS, alpha=1e-2)

model1.compile(
    optimizer=keras.optimizers.Adam(learning_rate=lr),
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=[
        keras.metrics.CategoricalAccuracy(name="accuracy"),
        keras.metrics.TopKCategoricalAccuracy(k=5, name="top5_acc"),
    ],
)

cb = [
    keras.callbacks.ModelCheckpoint("/content/task1_best.keras", save_best_only=True, monitor="val_accuracy", mode="max", verbose=1),
    keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=15, restore_best_weights=True),
]

history = model1.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=cb, verbose=1)

print("Eval:", model1.evaluate(val_ds, verbose=1))

final_path = "/content/task1_custom_final.keras"
model1.save(final_path)

from google.colab import files
files.download("/content/task1_best.keras")
files.download(final_path)


Model: "task1_scratch"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_25      │ (None, 160, 160,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ aug (Sequential)    │ (None, 160, 160,  │          0 │ input_layer_25[0… │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_7         │ (None, 160, 160,  │          0 │ aug[0][0]         │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 80, 80,    │        864 │ rescaling_7[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_ln             │ (None, 80, 80,    │         64 │ conv2d_3[0][0]    │
│ (LayerNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_relu (ReLU)    │ (None, 80, 80,    │          0 │ stem_ln[0][0]     │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b1_dw               │ (None, 80, 80,    │        288 │ stem_relu[0][0]   │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b1_a_ln             │ (None, 80, 80,    │         64 │ b1_dw[0][0]       │
│ (LayerNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b1_a_relu (ReLU)    │ (None, 80, 80,    │          0 │ b1_a_ln[0][0]     │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b1_pw (Conv2D)      │ (None, 80, 80,    │      2,048 │ b1_a_relu[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b1_b_ln             │ (None, 80, 80,    │        128 │ b1_pw[0][0]       │
│ (LayerNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b1_b_relu (ReLU)    │ (None, 80, 80,    │          0 │ b1_b_ln[0][0]     │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b2_dw               │ (None, 40, 40,    │        576 │ b1_b_relu[0][0]   │
│ (DepthwiseConv2D)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b2_a_ln             │ (None, 40, 40,    │        128 │ b2_dw[0][0]       │
│ (LayerNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b2_a_relu (ReLU)    │ (None, 40, 40,    │          0 │ b2_a_ln[0][0]     │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b2_pw (Conv2D)      │ (None, 40, 40,    │      8,192 │ b2_a_relu[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b2_b_ln             │ (None, 40, 40,    │        256 │ b2_pw[0][0]     

 Total params: 205,642 (803.29 KB)

 Trainable params: 205,642 (803.29 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/80
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - accuracy: 0.1037 - loss: 3.1559 - top5_acc: 0.5436
Epoch 1: val_accuracy improved from -inf to 0.10000, saving model to /content/task1_best.keras
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 209ms/step - accuracy: 0.1049 - loss: 3.1491 - top5_acc: 0.5444 - val_accuracy: 0.1000 - val_loss: 2.5532 - val_top5_acc: 0.5000
Epoch 2/80
12/13 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.0710 - loss: 2.7977 - top5_acc: 0.4458
Epoch 2: val_accuracy did not improve from 0.10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - accuracy: 0.0687 - loss: 2.7984 - top5_acc: 0.4407 - val_accuracy: 0.1000 - val_loss: 2.4218 - val_top5_acc: 0.5200
Epoch 3/80
12/13 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.0543 - loss: 2.7466 - top5_acc: 0.4413
Epoch 3: val_accuracy did not improve from 0.10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - accuracy: 0.0586 - loss: 2.7358 - top5_acc: 0.4419 - val_accuracy: 0.1000 - val_loss: 2.3231 - val_top5_acc: 0.4800
Epoch 4/80
12/1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>